In [20]:
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dropout,Input,Flatten,Dense,MaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [21]:
physical_devices =tf.config.list_physical_devices('GPU')

In [22]:
batchsize=8

In [23]:
train_datagen=ImageDataGenerator(rescale=1./255,rotation_range=0.2,shear_range=0.2,
                              zoom_range=0.2,width_shift_range=0.2,
                                 height_shift_range=0.2,validation_split=0.2)

In [24]:
train_data=train_datagen.flow_from_directory(r'/Users/pritamsingh/Downloads/pd/train',
                                             target_size=(80,80),batch_size=8,class_mode='categorical',subset='training')
validation_data=train_datagen.flow_from_directory(r'/Users/pritamsingh/Downloads/pd/train',
                                            target_size=(80,80),batch_size=8,class_mode='categorical',subset='validation')

Found 64327 images belonging to 2 classes.
Found 16081 images belonging to 2 classes.


In [25]:
test_datagen=ImageDataGenerator(rescale=1./255)

In [26]:
test_data=test_datagen.flow_from_directory(r'/Users/pritamsingh/Downloads/pd/test',
                                           target_size=(80,80),batch_size=8,class_mode='categorical')

Found 4490 images belonging to 2 classes.


In [27]:
base_model= InceptionV3(include_top=False,weights='imagenet',input_tensor=Input(shape=(80,80,3)))
head_model=base_model.output

In [28]:
base_model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 80, 80, 3)]  0           []                               
                                                                                                  
 conv2d_94 (Conv2D)             (None, 39, 39, 32)   864         ['input_2[0][0]']                
                                                                                                  
 batch_normalization_94 (BatchN  (None, 39, 39, 32)  96          ['conv2d_94[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_94 (Activation)     (None, 39, 39, 32)   0           ['batch_normalization_

In [29]:
head_model=Flatten()(head_model)
head_model=Dense(64,activation='relu')(head_model)
head_model=Dropout(0.5)(head_model)
head_model=Dense(2,activation='softmax')(head_model)

model=Model(inputs=base_model.input,outputs=head_model)

for layer in base_model.layers:
    layer.trainable=False

In [30]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 80, 80, 3)]  0           []                               
                                                                                                  
 conv2d_94 (Conv2D)             (None, 39, 39, 32)   864         ['input_2[0][0]']                
                                                                                                  
 batch_normalization_94 (BatchN  (None, 39, 39, 32)  96          ['conv2d_94[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_94 (Activation)     (None, 39, 39, 32)   0           ['batch_normalization_94[0]

In [31]:
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping,ReduceLROnPlateau

In [32]:
checkpoint=ModelCheckpoint(r'/Users/pritamsingh/Downloads/models/model.h5',monitor='val_loss',save_best_only=True,verbose=3)

earlystop=EarlyStopping(monitor='val_loss',patience=7,verbose=3,restore_best_weights=True)
learning_rate=ReduceLROnPlateau(monitor='val_loss',patience=3,verbose=3)
callbacks=[checkpoint,earlystop,learning_rate]

In [33]:
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array

In [34]:
import keras
from PIL import Image
from sklearn.preprocessing import LabelEncoder


In [ ]:
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])
model.fit_generator(train_data,steps_per_epoch=train_data.samples//batchsize,
                   validation_data=validation_data,
                   validation_steps=validation_data.samples//batchsize,
                   callbacks=callbacks,epochs=5)

Epoch 1/5


/var/folders/w2/dw553ptx4cg7hns5mxx_x5rc0000gn/T/ipykernel_5205/1839759979.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_data,steps_per_epoch=train_data.samples//batchsize,


8040/8040 [==============================] - ETA: 0s - loss: 0.2063 - accuracy: 0.9195
Epoch 1: val_loss improved from inf to 0.17361, saving model to /Users/pritamsingh/Downloads/models/model.h5
8040/8040 [==============================] - 756s 93ms/step - loss: 0.2063 - accuracy: 0.9195 - val_loss: 0.1736 - val_accuracy: 0.9325 - lr: 0.0010
Epoch 2/5
8040/8040 [==============================] - ETA: 0s - loss: 0.1766 - accuracy: 0.9320
Epoch 2: val_loss did not improve from 0.17361
8040/8040 [==============================] - 747s 93ms/step - loss: 0.1766 - accuracy: 0.9320 - val_loss: 0.1797 - val_accuracy: 0.9264 - lr: 0.0010
Epoch 3/5
8040/8040 [==============================] - ETA: 0s - loss: 0.1690 - accuracy: 0.9350
Epoch 3: val_loss did not improve from 0.17361
8040/8040 [==============================] - 752s 94ms/step - loss: 0.1690 - accuracy: 0.9350 - val_loss: 0.1977 - val_accuracy: 0.9159 - lr: 0.0010
Epoch 4/5
8040/8040 [==============================] - ETA: 0s - loss